In [15]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm

In [16]:
def get_connections(url):

  r=requests.get(url)
  soup=BeautifulSoup(r.text)

  nodes=[]
  errors=[]
  for line in soup.find_all('a'):
    if 'href="/wiki/' in str(line):
      try:
        link=str(line).split('href="')[1].split('"')[0]
        title=str(line).split('title="')[1].split('"')[0]
        if ':' not in link:
          link='https://en.wikipedia.org'+link
          nodes.append([link,title])
        else:
          pass
      except:
        errors.append(line)

    df=pd.DataFrame(nodes,columns=['link','title']).drop_duplicates()
    links=df['link'].tolist()
    titles=df['title'].tolist()

  return links,titles,errors

In [17]:
links,titles,_=get_connections('https://en.wikipedia.org/wiki/PageRank')

In [18]:
connection_df=pd.DataFrame(np.zeros((len(links),len(links))),index=titles,columns=titles)

In [19]:
columns=connection_df.columns.tolist()
for link,title in tqdm(zip(links,titles)):
  connection_links,connection_titles,_=get_connections(link)
  for connection_link,connection_title in zip(connection_links,connection_titles):
    if connection_title in columns:
      connection_df.at[title,connection_title]=1

136it [03:38,  1.61s/it]


In [22]:
connection_df

,Visit the main page [z],View the content page [c],Algorithm,Google Search,Webpages,Search engine,Larry Page,Network theory,Weighting,Hyperlink,...,Wayback Machine,Forbes,ISBN (identifier),Bibcode (identifier),PMID (identifier),Barry Schwartz (technologist),Search Engine Land,PMC (identifier),Lesley Ward,Amy Langville
Visit the main page [z],1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
View the content page [c],1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Algorithm,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Google Search,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Webpages,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Barry Schwartz (technologist),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Search Engine Land,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PMC (identifier),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
Lesley Ward,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
connection_df.to_excel('connection_matrix.xlsx')